In [1]:
import pandas as pd
import os
from tqdm import tqdm
import shutil

In [2]:
new_image_path = r'//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated'
new_folder_names = [
    'no_contact',
    'tongue',
    'paw',
]
new_image_path

'//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated'

In [3]:
existing_image_path = r'//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/models/three_class_model'
existing_folder_names = [
    'train',
    'test',
    'validate',
]
existing_image_path

'//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/models/three_class_model'

In [4]:
new_data = []
for folder_name in new_folder_names:
    df = pd.DataFrame({'filename':os.listdir(os.path.join(new_image_path,folder_name))})
    df['folder'] = folder_name
    new_data.append(df)
new_data = pd.concat(new_data)

In [5]:
new_data

,filename,folder
0,oeid_956903412__frame_91093.png,no_contact
1,oeid_956903412__frame_89967.png,no_contact
2,oeid_956903412__frame_116114.png,no_contact
3,oeid_956903412__frame_123426.png,no_contact
4,oeid_956903412__frame_123366.png,no_contact
...,...,...
2,oeid_846490568__frame_117771.png,paw
3,oeid_956903412__frame_81653.png,paw
4,Thumbs.db,paw
5,oeid_846490568__frame_85731.png,paw


In [15]:
existing_filenames = []
for foldername in existing_folder_names:
    for category in new_folder_names:
        existing_filenames += os.listdir(os.path.join(existing_image_path, foldername, category))

In [18]:
len(existing_filenames)

11316

In [8]:
new_data = new_data.sample(len(new_data)).reset_index()
new_data['set'] = None
new_data['label'] = None

In [9]:
def label(folder):
    if folder.startswith('tongue'):
        return 'tongue'
    elif folder.startswith('paw'):
        return 'paw'
    else:
        return 'no_contact'

In [10]:
new_data.loc[:0.8*len(new_data),'set'] = 'train'
new_data.loc[0.8*len(new_data):0.9*len(new_data),'set'] = 'validate'
new_data.loc[0.9*len(new_data):,'set'] = 'test'
new_data['label'] = new_data['folder'].map(label)

In [11]:
new_data

,index,filename,folder,set,label
0,34,oeid_956903412__frame_69835.png,no_contact,train,no_contact
1,31,oeid_956903412__frame_103637.png,no_contact,train,no_contact
2,187,oeid_956903412__frame_80186.png,no_contact,train,no_contact
3,199,oeid_846490568__frame_2871.png,no_contact,train,no_contact
4,80,oeid_956903412__frame_105038.png,no_contact,train,no_contact
...,...,...,...,...,...
332,218,oeid_956903412__frame_11961.png,no_contact,test,no_contact
333,259,oeid_956903412__frame_91142.png,no_contact,test,no_contact
334,281,oeid_956903412__frame_27459.png,no_contact,test,no_contact
335,169,oeid_956903412__frame_48137.png,no_contact,test,no_contact


In [12]:
new_data['label'].value_counts()

no_contact    315
tongue         15
paw             7
Name: label, dtype: int64

In [13]:
new_data['set'].value_counts()

train       270
validate     34
test         33
Name: set, dtype: int64

In [14]:
data_dst = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/models/three_class_model'
for idx,row in tqdm(new_data.iterrows()):
    src = os.path.join(new_image_path,row['folder'],row['filename'])
    dst = os.path.join(existing_image_path,row['set'],str(row['label']))
    if os.path.exists(dst) == False:
        os.mkdir(dst)
    if row['filename'] not in existing_filenames:
        shutil.copy(src,dst)
    else:
        print('did not copy {}, already exists'.format(src))

19it [00:00, 82.97it/s]

did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/no_contact/oeid_956903412__frame_103637.png, already exists
did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/no_contact/oeid_956903412__frame_80186.png, already exists
did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/no_contact/oeid_956903412__frame_105038.png, already exists
did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/no_contact/oeid_956903412__frame_111690.png, already exists
did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/no_contact/oeid_956903412__frame_89188.png, already exists
did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manuall

38it [00:00, 63.41it/s]

did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/no_contact/oeid_956903412__frame_89581.png, already exists
did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/no_contact/oeid_956903412__frame_106721.png, already exists
did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/no_contact/oeid_956903412__frame_116004.png, already exists
did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/no_contact/oeid_956903412__frame_61065.png, already exists
did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/no_contact/oeid_956903412__frame_111034.png, already exists
did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manuall

50it [00:00, 50.50it/s]

...

316it [00:05, 57.09it/s]

did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/no_contact/oeid_956903412__frame_91007.png, already exists
did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/tongue/oeid_846490568__frame_43646.png, already exists
did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/no_contact/oeid_956903412__frame_69914.png, already exists
did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/no_contact/oeid_956903412__frame_91098.png, already exists
did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/no_contact/oeid_956903412__frame_111768.png, already exists
did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_anno

324it [00:05, 60.27it/s]

did not copy //allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/manually_annotated/no_contact/oeid_956903412__frame_117645.png, already exists


337it [00:05, 46.61it/s]


In [27]:
target_path = '/ibs-dougo-ux1/root/local'
os.path.exists(target_path)

False

In [ ]:
from dirsync import sync
source_path = data_dst = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/models/three_class_model'
target_path = '/Give/Target/Folder/Here'

sync(source_path, target_path, 'sync') #for syncing one way
# sync(target_path, source_path, 'sync') #for syncing the opposite way